In [7]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [8]:
data = pd.read_csv('train.csv')
data.set_index('PassengerId', inplace=True)
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [9]:
data_ = data['Ticket'].apply(lambda x: x.split(' '))
data['ticket_1'] = data_.apply(lambda x: x[0] if len(x)==2 else np.nan if len(x)==1 else x[0]+x[1])
data['ticket_2'] = data_.apply(lambda x: x[-1])
data['ticket_1'][data['ticket_2'].apply(lambda x: x if x.isnumeric() else np.nan).isnull()] = 'Line'
data['ticket_1'][data['ticket_2'].apply(lambda x: x if x.isnumeric() else np.nan).isnull()] = np.nan

/var/folders/86/zm_2nrj52293tl8svbmbng1w0000gn/T/ipykernel_10495/3183689096.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ticket_1'][data['ticket_2'].apply(lambda x: x if x.isnumeric() else np.nan).isnull()] = 'Line'
/var/folders/86/zm_2nrj52293tl8svbmbng1w0000gn/T/ipykernel_10495/3183689096.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ticket_1'][data['ticket_2'].apply(lambda x: x if x.isnumeric() else np.nan).isnull()] = np.nan


In [10]:
data_ = data['ticket_1'].apply(lambda x: x.split('/') if type(x)==str else x)
data_ = data_.apply(pd.Series).stack()
data_ = DataFrame(data_)
one_hot_1 = OneHotEncoder(handle_unknown='ignore')
data__ = one_hot_1.fit_transform(data_)
data_ = pd.DataFrame(data__.toarray(), index=data_.index)
data_ = data_.groupby(level=0).sum()
data_.columns = one_hot_1.get_feature_names_out()
data_

,x0_4,x0_4.,x0_5,x0_5.,x0_A,x0_A.,x0_A.4.,x0_A.5.,x0_A4.,x0_AH,...,x0_SC,x0_SCO,x0_SO,x0_SOTON,x0_STON,x0_SW,x0_W,x0_W.,x0_W.E.P.,x0_WE
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
data = pd.merge(data, data_, left_index=True, right_index=True, how='outer')
data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,x0_SC,x0_SCO,x0_SO,x0_SOTON,x0_STON,x0_SW,x0_W,x0_W.,x0_W.E.P.,x0_WE
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
data_ = data['Name'].apply(lambda x: x.split('(')[0])
data_ = data_.apply(lambda x: x.split(' '))
def f(l):
    while l[-1]=='':
        l.pop()
    return l[-1]
def g(string):
    ans = str()
    for char in string:
        if char.isalpha():
            ans+=char
    return ans
data_ = data_.apply(f)
data_ = data_.apply(g)
data_

PassengerId
1       Harris
2      Bradley
3        Laina
4        Heath
5        Henry
        ...   
887     Juozas
888      Edith
889     Carrie
890     Howell
891    Patrick
Name: Name, Length: 891, dtype: object

In [13]:
data = pd.merge(data, data_, left_index=True, right_index=True, how='outer')
data

,Survived,Pclass,Name_x,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,x0_SCO,x0_SO,x0_SOTON,x0_STON,x0_SW,x0_W,x0_W.,x0_W.E.P.,x0_WE,Name_y
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Harris
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bradley
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Laina
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heath
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Henry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juozas
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Edith
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Carrie


In [14]:
data_ = data.drop(['Ticket','Name_x', 'ticket_1'], axis=1).select_dtypes(exclude=['number'])
one_hot_2 = OneHotEncoder(handle_unknown='ignore')
data__ = one_hot_2.fit_transform(data_).toarray()
data_ = DataFrame(data__, index=data_.index, columns=one_hot_2.get_feature_names_out())
data = pd.merge(data, data_, left_index=True, right_index=True, how='outer')
data

,Survived,Pclass,Name_x,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Name_y_Willingham,Name_y_Wills,Name_y_Wilson,Name_y_Winnie,Name_y_Wyckoff,Name_y_Yoto,Name_y_Youssef,Name_y_Yousseff,Name_y_Zebley,Name_y_of
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
data_ = data.select_dtypes(['number'])
data_.fillna(0, inplace=True)
data_

,Survived,Pclass,Age,SibSp,Parch,Fare,x0_4,x0_4.,x0_5,x0_5.,...,Name_y_Willingham,Name_y_Wills,Name_y_Wilson,Name_y_Winnie,Name_y_Wyckoff,Name_y_Yoto,Name_y_Youssef,Name_y_Yousseff,Name_y_Zebley,Name_y_of
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,22.0,1,0,7.2500,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,38.0,1,0,71.2833,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,26.0,0,0,7.9250,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,35.0,1,0,53.1000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,3,35.0,0,0,8.0500,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,27.0,0,0,13.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,1,1,19.0,0,0,30.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,0,3,0.0,1,2,23.4500,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
y = data_['Survived']
X = data_.drop(['Survived'], axis=1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=999)

In [19]:
cur_ans = 0
best_n = 0
best_d = 0
for estimators in range(5, 200, 5):
    for depth in range(10, 200, 10):
        rf_model = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=1)
        rf_model.fit(X_train, y_train)
        y_pred  = rf_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        if accuracy > cur_ans:
            cur_ans = accuracy
            best_n = estimators
            best_d = depth
cur_ans

0.8379888268156425

In [21]:
print(best_n,best_d)

50 70


In [22]:
rf_model = RandomForestClassifier(n_estimators=best_n, max_depth=best_d, random_state=1)
rf_model.fit(X_train, y_train)
y_pred  = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8379888268156425

In [24]:
task = pd.read_csv('test.csv')
task.set_index('PassengerId', inplace=True)
task

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [25]:
task_ = task['Ticket'].apply(lambda x: x.split(' '))
task['ticket_1'] = task_.apply(lambda x: x[0] if len(x)==2 else np.nan if len(x)==1 else x[0]+x[1])
task['ticket_2'] = task_.apply(lambda x: x[-1])
task['ticket_1'][task['ticket_2'].apply(lambda x: x if x.isnumeric() else np.nan).isnull()] = 'Line'
task['ticket_1'][task['ticket_2'].apply(lambda x: x if x.isnumeric() else np.nan).isnull()] = np.nan

/var/folders/86/zm_2nrj52293tl8svbmbng1w0000gn/T/ipykernel_10495/784241794.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task['ticket_1'][task['ticket_2'].apply(lambda x: x if x.isnumeric() else np.nan).isnull()] = 'Line'
/var/folders/86/zm_2nrj52293tl8svbmbng1w0000gn/T/ipykernel_10495/784241794.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  task['ticket_1'][task['ticket_2'].apply(lambda x: x if x.isnumeric() else np.nan).isnull()] = np.nan


In [28]:
task_ = task['ticket_1'].apply(lambda x: x.split('/') if type(x)==str else x)
task_ = task_.apply(pd.Series).stack()
task_ = DataFrame(task_)
task__ = one_hot_1.transform(task_)
task_ = pd.DataFrame(task__.toarray(), index=task_.index)
task_ = task_.groupby(level=0).sum()
task_.columns = one_hot_1.get_feature_names_out()
task_

,x0_4,x0_4.,x0_5,x0_5.,x0_A,x0_A.,x0_A.4.,x0_A.5.,x0_A4.,x0_AH,...,x0_SC,x0_SCO,x0_SO,x0_SOTON,x0_STON,x0_SW,x0_W,x0_W.,x0_W.E.P.,x0_WE
PassengerId,,,,,,,,,,,,,,,,,,,,,
901,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
task = pd.merge(task, task_, left_index=True, right_index=True, how='outer')
task

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,x0_SC,x0_SCO,x0_SO,x0_SOTON,x0_STON,x0_SW,x0_W,x0_W.,x0_W.E.P.,x0_WE
PassengerId,,,,,,,,,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
task_ = task['Name'].apply(lambda x: x.split('(')[0])
task_ = task_.apply(lambda x: x.split(' '))
task_ = task_.apply(f)
task_ = task_.apply(g)
task_

PassengerId
892         James
893         James
894       Francis
895        Albert
896     Alexander
          ...    
1305        Woolf
1306      Fermina
1307    Sivertsen
1308    Frederick
1309            J
Name: Name, Length: 418, dtype: object

In [33]:
task = pd.merge(task, task_, left_index=True, right_index=True, how='outer')
task

,Pclass,Name_x,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,x0_SCO,x0_SO,x0_SOTON,x0_STON,x0_SW,x0_W,x0_W.,x0_W.E.P.,x0_WE,Name_y
PassengerId,,,,,,,,,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,James
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,James
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Francis
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albert
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alexander
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Woolf
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fermina
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Sivertsen


In [34]:
task_ = task.drop(['Ticket','Name_x', 'ticket_1'], axis=1).select_dtypes(exclude=['number'])
task__ = one_hot_2.transform(task_).toarray()
task_ = DataFrame(task__, index=task_.index, columns=one_hot_2.get_feature_names_out())
task = pd.merge(task, task_, left_index=True, right_index=True, how='outer')
task

,Pclass,Name_x,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,Name_y_Willingham,Name_y_Wills,Name_y_Wilson,Name_y_Winnie,Name_y_Wyckoff,Name_y_Yoto,Name_y_Youssef,Name_y_Yousseff,Name_y_Zebley,Name_y_of
PassengerId,,,,,,,,,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
task_ = task.select_dtypes(['number'])
task_.fillna(0, inplace=True)
task_

,Pclass,Age,SibSp,Parch,Fare,x0_4,x0_4.,x0_5,x0_5.,x0_A,...,Name_y_Willingham,Name_y_Wills,Name_y_Wilson,Name_y_Winnie,Name_y_Wyckoff,Name_y_Yoto,Name_y_Youssef,Name_y_Yousseff,Name_y_Zebley,Name_y_of
PassengerId,,,,,,,,,,,,,,,,,,,,,
892,3,34.5,0,0,7.8292,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
893,3,47.0,1,0,7.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
894,2,62.0,0,0,9.6875,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
895,3,27.0,0,0,8.6625,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
896,3,22.0,1,1,12.2875,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,0.0,0,0,8.0500,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1306,1,39.0,0,0,108.9000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1307,3,38.5,0,0,7.2500,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rf_model = RandomForestClassifier(n_estimators=best_n, max_depth=best_d, random_state=1)
rf_model.fit(X, y)
task_pred  = rf_model.predict(X_test)